In [1]:
from dell_runtime import DellRuntimeProvider

In [2]:
RUNTIME_PROGRAM = """
import random

from qiskit import transpile
from qiskit.circuit.random import random_circuit

def prepare_circuits(backend):
    circuit = random_circuit(num_qubits=5, depth=4, measure=True,
                            seed=random.randint(0, 1000))
    return transpile(circuit, backend)

def main(backend, user_messenger, **kwargs):
    iterations = kwargs['iterations']
    interim_results = kwargs.pop('interim_results', {})
    final_result = kwargs.pop("final_result", {})
    for it in range(iterations):
        qc = prepare_circuits(backend)
        user_messenger.publish({"iteration": it, "interim_results": interim_results})
        backend.run(qc).result()

    user_messenger.publish(final_result, final=True)
"""

In [3]:
RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

PROGRAM_PREFIX = 'qiskit-test'

In [4]:
provider = DellRuntimeProvider()
provider.runtime.programs()


{}

In [5]:
program_id = provider.runtime.upload_program(RUNTIME_PROGRAM, metadata=RUNTIME_PROGRAM_METADATA)

In [6]:
provider.runtime.pprint_programs()

4af41424c6ac066c:
  Name: 4af41424c6ac066c
  Description: Qiskit test program
  Version: 0
  Creation date: 24/09/2021 14:22:31
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none


In [7]:
runtime_program = provider.runtime.program(program_id)
program_inputs = {
    "iterations": 10
}

In [8]:
job = provider.runtime.run(program_id, options=None, inputs=program_inputs)

DEBUG:dell_runtime.emulation_executor:creating temp directory at /tmp/tmpyrd2h0nh
DEBUG:dell_runtime.emulation_executor:finished writing to /tmp/tmpyrd2h0nh/program.py
DEBUG:dell_runtime.emulation_executor:finished writing to /tmp/tmpyrd2h0nh/params.json
DEBUG:dell_runtime.emulation_executor:finished writing to /tmp/tmpyrd2h0nh/executor.py
DEBUG:dell_runtime.emulation_executor:starting ['/home/geoff/miniconda3/envs/qruntime/bin/python3', '/tmp/tmpyrd2h0nh/executor.py']
DEBUG:dell_runtime.emulation_executor:finished executing ['/home/geoff/miniconda3/envs/qruntime/bin/python3', '/tmp/tmpyrd2h0nh/executor.py']
DEBUG:dell_runtime.emulation_executor:stdout: using default backend: aer_simulator
exit

DEBUG:dell_runtime.emulation_executor:stderr: 
DEBUG:dell_runtime.emulation_executor:status: sent COMPLETED


In [9]:
job.status()

'Completed'